In [1]:
from models import base_res_net
import tensorflow as tf
from tensorflow import keras
import tensorflow.compat.v1 as tfc

In [3]:
# Some GPU setup
# for documentation about using gpus referto: https://www.tensorflow.org/install/pip#windows-wsl2

tf.keras.backend.clear_session()

device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
try:  # prevent a previous session from being alive
  sess.close() 
except:
  pass

tfc.enable_eager_execution()
gpu_options= tfc.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tfc.InteractiveSession(config=tfc.ConfigProto(gpu_options=gpu_options))

In [ ]:
INPUT_SHAPE = (218,145,1)

CONF = {
  "learning_rate": 0.0001,
  "batch_size": 32,
  "epochs": 85,
  "input_width": INPUT_SHAPE[0],
  "input_height": INPUT_SHAPE[1],
  "loss_function": "sparse_categorical_crossentropy",
}

In [6]:
base_model = base_res_net(INPUT_SHAPE, 10)
base_model.summary()
base_model.compile(loss=CONF.loss_function,
             optimizer=keras.optimizers.Adam(CONF.learning_rate),
             metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 145, 218, 1  0           []                               
                                )]                                                                
                                                                                                  
 sequential_1 (Sequential)      (None, 145, 218, 1)  0           ['input_2[0][0]']                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 73, 109, 16)  160         ['sequential_1[0][0]']           
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 73, 109, 16)  64         ['conv2d_4[0][0]']         

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 sequential (Sequential)        (None, 64, 64, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['sequential[0][0]']             
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                   